In [1]:
import pandas as pd
import time

In [2]:
with open('onegin.txt') as f:
  phrases = [s.replace('\t\t','') for s in f.read().split('\n') if s.startswith('\t\t')]

In [3]:
phrases

['He мысля гордый свет забавить,',
 'Вниманье дружбы возлюбя,',
 'Хотел бы я тебе представить',
 'Залог достойнее тебя,',
 'Достойнее души прекрасной,',
 'Святой исполненной мечты,',
 'Поэзии живой и ясной,',
 'Высоких дум и простоты;',
 'Но так и быть – рукой пристрастной',
 'Прими собранье пёстрых глав,',
 'Полусмешных, полупечальных,',
 'Простонародных, идеальных,',
 'Небрежный плод моих забав,',
 'Бессонниц, лёгких вдохновений,',
 'Незрелых и увядших лет,',
 'Ума холодных наблюдений',
 'И сердца горестных замет.',
 '«Мой дядя самых честных правил,',
 'Когда не в шутку занемог,',
 'Он уважать себя заставил',
 'И лучше выдумать не мог.',
 'Его пример другим наука;',
 'Но, боже мой, какая скука',
 'С больным сидеть и день и ночь,',
 'Не отходя ни шагу прочь!',
 'Какое низкое коварство',
 'Полуживого забавлять,',
 'Ему подушки поправлять,',
 'Печально подносить лекарство,',
 'Вздыхать и думать про себя:',
 'Когда же чёрт возьмёт тебя!»',
 'Так думал молодой повеса,',
 'Летя в пыли на п

In [4]:
phrases[:10]

['He мысля гордый свет забавить,',
 'Вниманье дружбы возлюбя,',
 'Хотел бы я тебе представить',
 'Залог достойнее тебя,',
 'Достойнее души прекрасной,',
 'Святой исполненной мечты,',
 'Поэзии живой и ясной,',
 'Высоких дум и простоты;',
 'Но так и быть – рукой пристрастной',
 'Прими собранье пёстрых глав,']

In [5]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [6]:
CHARS = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

In [7]:
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]

In [8]:
INDEX_TO_CHAR

['none',
 'е',
 'ю',
 'щ',
 'д',
 'л',
 'р',
 'б',
 'ч',
 'ё',
 'а',
 'ф',
 'п',
 'с',
 'я',
 'г',
 'з',
 'у',
 'й',
 'м',
 'ц',
 'и',
 'о',
 'в',
 'н',
 'ы',
 'т',
 'ъ',
 'к',
 'ш',
 'х',
 'ь',
 'ж',
 'э',
 ' ']

In [9]:
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [10]:
CHAR_TO_INDEX

{'none': 0,
 'е': 1,
 'ю': 2,
 'щ': 3,
 'д': 4,
 'л': 5,
 'р': 6,
 'б': 7,
 'ч': 8,
 'ё': 9,
 'а': 10,
 'ф': 11,
 'п': 12,
 'с': 13,
 'я': 14,
 'г': 15,
 'з': 16,
 'у': 17,
 'й': 18,
 'м': 19,
 'ц': 20,
 'и': 21,
 'о': 22,
 'в': 23,
 'н': 24,
 'ы': 25,
 'т': 26,
 'ъ': 27,
 'к': 28,
 'ш': 29,
 'х': 30,
 'ь': 31,
 'ж': 32,
 'э': 33,
 ' ': 34}

In [11]:
import torch

In [12]:
MAX_LEN = 30

In [13]:
X = torch.zeros((len(text), MAX_LEN), dtype=int)

In [14]:
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [15]:
X[0:1]

tensor([[ 0,  0, 34, 19, 25, 13,  5, 14, 34, 15, 22,  6,  4, 25, 18, 34, 13, 23,
          1, 26, 34, 16, 10,  7, 10, 23, 21, 26, 31,  0]])

In [16]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.RNN(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [17]:
class NetworkGRU(torch.nn.Module):

    def __init__(self):
        super(NetworkGRU, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.GRU(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [18]:
class NetworkLSTM(torch.nn.Module):

    def __init__(self):
        super(NetworkLSTM, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.lstm = torch.nn.LSTM(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.lstm(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.lstm(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [39]:
class NetworkUber(torch.nn.Module):

    def __init__(self):
        super(NetworkUber, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.GRU(35, 128, batch_first=True, num_layers=3)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [20]:
model = Network()

In [21]:
model.forward(X[0:1])[0].shape

torch.Size([1, 30, 35])

In [22]:
def generate_sentence(mdl):
    sentence = ['я', ' ', 'в', 'а', 'с']
    #sentence = ['а', 'х', ' ', 'н', 'о', 'ж', 'к', 'и']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        if i == 0:
            result, state = mdl.forward(X)
        else:
            result, state = mdl.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
    print(''.join(sentence))

In [23]:
generate_sentence(model)

я василхзаияаиёииёбииии ииии виии и


In [32]:
def learning_loop(mdl, epochs=2000, optimizer=None):
    criterion = torch.nn.CrossEntropyLoss()
    for ep in range(epochs):
        start = time.time()
        train_loss = 0.
        train_passed = 0

        for i in range(int(len(X) / 100)):
            batch = X[i * 100:(i + 1) * 100]
            X_batch = batch[:, :-1]
            Y_batch = batch[:, 1:].flatten()

            optimizer.zero_grad()
            answers, _ = mdl.forward(X_batch)
            answers = answers.view(-1, len(INDEX_TO_CHAR))
            loss = criterion(answers, Y_batch)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()
            train_passed += 1

        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        generate_sentence(mdl)

In [35]:
learning_loop(model, epochs=100, optimizer=torch.optim.SGD(model.parameters(), lr=.5))

Epoch 0. Time: 0.754, Train loss: 0.284
я вастсзсnoneххояааогткnoneкоуоnone й тnoneимпо
Epoch 1. Time: 0.645, Train loss: 0.284
я вастсзсnoneххояааогткnoneкоуоnone й тnoneимпо
Epoch 2. Time: 0.642, Train loss: 0.284
я вастсзсnoneххояааогткnoneкоуоnone й йnoneимпnone
Epoch 3. Time: 0.632, Train loss: 0.284
я вастсзсnoneххояааогткnoneкоуоnone й тnoneимпо
Epoch 4. Time: 0.723, Train loss: 0.284
я вастсзсnoneххояааогткnoneкоаоnone ййнnonenoneртё
Epoch 5. Time: 0.676, Train loss: 0.284
я вастсзсnoneххояааогткnoneкоуоnoneой тnoneиnonenoneь
Epoch 6. Time: 0.734, Train loss: 0.283
я вастсзсnoneххояааогткnoneкоуоnone й йnoneимпnone
Epoch 7. Time: 0.683, Train loss: 0.283
я вастсзсnoneххояааогткnoneкоуоnoneой тnoneиnonenoneь
Epoch 8. Time: 0.728, Train loss: 0.283
я вастсзсnoneххояааогткnoneкоуоnoneой тnoneиnonenoneь
Epoch 9. Time: 0.726, Train loss: 0.283
я вастсзсnoneххояааогткnoneкоуоnoneой тnoneиnonenoneь
Epoch 10. Time: 0.835, Train loss: 0.283
я вастсзсnoneххояааогткnoneкоаоnoneоййнnoneгno

In [36]:
model = NetworkGRU()
learning_loop(model, epochs=100, optimizer=torch.optim.Adam(model.parameters(), lr=.5))

Epoch 0. Time: 1.264, Train loss: 21.416
я васакррйздееnoneаеееnonenoneдддnonenoneеееnonenoneдддnone
Epoch 1. Time: 1.408, Train loss: 14.819
я васянаnoneя  о  о оо ооооооооооооооо
Epoch 2. Time: 1.433, Train loss: 9.817
я васора ьййс   ьньпп ь ооь пвв ьоо
Epoch 3. Time: 1.575, Train loss: 6.739
я вас соnoneтногnoneоггоnoneгоооnoneин бnonenoneнннnonenone
Epoch 4. Time: 1.384, Train loss: 6.158
я васгжо тоаеnoneецnonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 5. Time: 1.390, Train loss: 7.595
я васгхыnone оnonenonenoneо nonenonenone nonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 6. Time: 1.356, Train loss: 8.712
я васвшыстыуnoneтрnoneюnonenoneнnonenonenonenoneаnonenonenonenoneкnonenonenonenoneн
Epoch 7. Time: 1.370, Train loss: 9.655
я васябnonenonenoneраnonenonenoneоnonenonenonenoneрnonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 8. Time: 1.362, Train loss: 9.455
я васххекьоодояnonenoneоnonenonenonenonenonenonenonen

In [37]:
model = NetworkLSTM()
learning_loop(model, epochs=100, optimizer=torch.optim.SGD(model.parameters(), lr=.5))

Epoch 0. Time: 1.202, Train loss: 3.074
я васnone nonenone nonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 1. Time: 1.112, Train loss: 2.745
я вас                              
Epoch 2. Time: 1.104, Train loss: 2.655
я вас     ооооо      ооооо      ооо
Epoch 3. Time: 1.379, Train loss: 2.605
я вас оо  о  оо оо  о   о ооо о    
Epoch 4. Time: 1.325, Train loss: 2.568
я вас оо оо  о  оо оо  о  оо оо  о 
Epoch 5. Time: 1.244, Train loss: 2.534
я вас оо оо  н  нооооо      ооооон 
Epoch 6. Time: 1.339, Train loss: 2.501
я вас оо оо  н  нооооо     нооооо  
Epoch 7. Time: 1.163, Train loss: 2.469
я вас оо оо  в  ввооооо     нвооооо
Epoch 8. Time: 1.071, Train loss: 2.438
я вас оо оол вл  ооооон     ооооов 
Epoch 9. Time: 1.110, Train loss: 2.408
я вас со ооо сл  втооооо     ввсооо
Epoch 10. Time: 1.125, Train loss: 2.380
я вас сососолт    аоооосл     ооооо
Epoch 11. Time: 1.091, Train loss: 2.353
я вас постсолтое  а  нссспоеето    
Ep

In [42]:
model = NetworkUber()
learning_loop(model, epochs=100, optimizer=torch.optim.SGD(model.parameters(), lr=.5))

Epoch 0. Time: 1.515, Train loss: 3.003
я васnone    nonenonenone   nonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 1. Time: 1.502, Train loss: 2.714
я вас  е   еееееее  nonenonenone   nonenonenonenone  nonenonenone
Epoch 2. Time: 1.540, Train loss: 2.632
я вас ооооее  nonenonenone о nonenonenonenoneо nonenonenonenonenonenonenonenonenone
Epoch 3. Time: 1.455, Train loss: 2.600
я вас оооооое nonenonenonenone nonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 4. Time: 1.597, Train loss: 2.576
я вас оооооое nonenonenonenone nonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 5. Time: 1.544, Train loss: 2.560
я вас оооооое nonenonenonenoneоnonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 6. Time: 1.978, Train loss: 2.545
я вас оооооое nonenonenonenoneеnonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 7. Time: 1.813, Train loss: 2.533
я вас оооооое nonenonenonenoneеnonenoneno